#### Лабораторная работа № 3. Разработка нейросетевых функций. Операция Convolution Transpose
#### БВТ2003 Суроп Максим Андреевич

In [64]:
import torch
import torch.nn.functional as F

In [65]:
import torch.nn as nn
import torch.nn.functional as F

class ConvTranspose(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, 
                 output_padding=0, groups=1, bias=True, dilation=1, padding_mode='zeros'):
        super(ConvTranspose, self).__init__()

        # Сохранение параметров
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.output_padding = output_padding
        self.dilation = dilation
        self.groups = groups
        self.bias = bias
        self.padding_mode = padding_mode

        # Инициализация параметров
        self.weight = nn.Parameter(torch.rand(out_channels, in_channels, *kernel_size))
        self.bias_param = nn.Parameter(torch.rand(out_channels)) if bias else None

    def forward(self, input):
        # Проверка режима паддинга
        if self.padding_mode != 'zeros':
            raise ValueError('Поддерживается только режим паддинга "zeros" в ConvTranspose2d')

        # Инвертирование пространственных размеров ядра для ConvTranspose2d
        filter_for_transpose = torch.flip(self.weight, [2, 3])

        # Операция ConvTranspose2d
        out_tensor = F.conv_transpose2d(
            input, filter_for_transpose, bias=self.bias_param,
            stride=self.stride, padding=self.padding, output_padding=self.output_padding,
            dilation=self.dilation, groups=1  # groups=1, так как в текущей реализации нет поддержки групп
        )

        # Применение паддинга для output_padding
        if self.output_padding > 0:
            pad_func = nn.ConstantPad2d((0, self.output_padding, 0, self.output_padding), 0)
            out_tensor = pad_func(out_tensor)

        # Обрезка результата по краям
        out_tensor = out_tensor[:, :, self.padding:out_tensor.shape[2] - self.padding,
                                   self.padding:out_tensor.shape[3] - self.padding]

        # Округление значений с повышенной точностью
        rounded_result = torch.round(out_tensor * 1e9) / 1e9

        return rounded_result


In [66]:
def test_conv_transpose():
    conv_transpose = ConvTranspose(
        in_channels=1, out_channels=1, kernel_size=(3, 3),
        stride=2, padding=1, output_padding=1
    )
    x = torch.ones((1, 1, 4, 4))  # Входные данные
    result = conv_transpose.forward(x)
    print("Тестовый пример:")
    print(f"Input:\n{x}\n")
    print(f"Output:\n{result}")

    # тестирование с использованием torch.nn.functional.conv_transpose2d
    filter_for_transpose = torch.flip(conv_transpose.weight, [2, 3])
    expected_result = F.conv_transpose2d(
        x, filter_for_transpose, bias=conv_transpose.bias_param,
        stride=conv_transpose.stride, padding=conv_transpose.padding,
        output_padding=conv_transpose.output_padding, dilation=conv_transpose.dilation, groups=1
    )
    print(f"\nОжидаемый результат:\n{expected_result}")


In [67]:
test_conv_transpose()

Тестовый пример:
Input:
tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]])

Output:
tensor([[[[3.5776, 1.2528, 3.5776, 1.2528, 3.5776, 1.2528, 1.9369],
          [1.9629, 1.2574, 1.9629, 1.2574, 1.9629, 1.2574, 1.4950],
          [3.5776, 1.2528, 3.5776, 1.2528, 3.5776, 1.2528, 1.9369],
          [1.9629, 1.2574, 1.9629, 1.2574, 1.9629, 1.2574, 1.4950],
          [3.5776, 1.2528, 3.5776, 1.2528, 3.5776, 1.2528, 1.9369],
          [1.9629, 1.2574, 1.9629, 1.2574, 1.9629, 1.2574, 1.4950],
          [2.3578, 1.1839, 2.3578, 1.1839, 2.3578, 1.1839, 1.6338]]]],
       grad_fn=<DivBackward0>)

Ожидаемый результат:
tensor([[[[1.2574, 1.9629, 1.2574, 1.9629, 1.2574, 1.9629, 1.2574, 1.4950],
          [1.2528, 3.5776, 1.2528, 3.5776, 1.2528, 3.5776, 1.2528, 1.9369],
          [1.2574, 1.9629, 1.2574, 1.9629, 1.2574, 1.9629, 1.2574, 1.4950],
          [1.2528, 3.5776, 1.2528, 3.5776, 1.2528, 3.5776, 1.2528, 1.9369],
          [1.257

#### Дополнительное задание :
#### Необходимо реализовать алгоритм работы транспонированной свертки через алгоритм двумерной свертки, реализованный в первой лабораторной

In [68]:
import torch
import numpy as np
from typing import Union

In [69]:
class Conv2DTranspose:
    def __init__(self, in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple] = 1) -> None:
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else kernel_size
        self.stride = (stride, stride) if isinstance(stride, int) else stride

        # Инициализация весов (транспонированное ядро свертки) и смещений
        self.weights = np.random.randn(in_channels, out_channels, *self.kernel_size)
        self.bias = np.zeros((out_channels,))

    def forward(self, input_tensor: np.ndarray) -> np.ndarray:
        batch_size, in_channels, input_height, input_width = input_tensor.shape
        kernel_size = self.kernel_size

        # Вычисление размеров результата транспонированной свертки
        stride_h, stride_w = self.stride
        output_height = (input_height - 1) * stride_h + kernel_size[0]
        output_width = (input_width - 1) * stride_w + kernel_size[1]

        # Инициализация результата транспонированной свертки
        result = np.zeros((batch_size, self.out_channels, output_height, output_width))

        # Выполнение транспонированной свертки
        for i in range(input_height):
            for j in range(input_width):
                window = input_tensor[:, :, i, j]
                result[:, :, i * stride_h:i * stride_h + kernel_size[0], j * stride_w:j * stride_w + kernel_size[1]] += \
                    np.tensordot(window, self.weights, axes=(1, 0))

        return result


In [70]:
def run_transpose_test(in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple],
                       padding: Union[int, tuple], dilation: int, groups: int, bias: bool, padding_mode: str) -> None:
    # Создание экземпляра Conv2DTranspose
    conv_transpose_layer = Conv2DTranspose(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)

    # Создание тензора для теста
    input_tensor = np.random.randn(1, in_channels, 10, 10)

    # Выполнение транспонированной свертки
    result_conv2d_transpose = conv_transpose_layer.forward(input_tensor)

    # Преобразование результатов в формат torch.Tensor для сравнения с PyTorch
    input_tensor_torch = torch.tensor(input_tensor)
    stride_h, stride_w = conv_transpose_layer.stride if isinstance(conv_transpose_layer.stride, tuple) else (conv_transpose_layer.stride, conv_transpose_layer.stride)

    output_tensor_torch = torch.nn.functional.conv_transpose2d(
        input_tensor_torch,
        torch.tensor(conv_transpose_layer.weights),
        bias=torch.tensor(conv_transpose_layer.bias),
        stride=(stride_h, stride_w),
        padding=padding,
        output_padding=stride_h - 1,
        dilation=dilation,
        groups=groups
    ).numpy()

    # Обрезание результатов из Conv2DTranspose, чтобы сделать размеры одинаковыми
    min_height = min(result_conv2d_transpose.shape[2], output_tensor_torch.shape[2])
    min_width = min(result_conv2d_transpose.shape[3], output_tensor_torch.shape[3])
    result_conv2d_transpose = result_conv2d_transpose[:, :, :min_height, :min_width]
    output_tensor_torch = output_tensor_torch[:, :, :min_height, :min_width]

    # Проверка совпадения результатов
    are_results_equal = np.allclose(result_conv2d_transpose, output_tensor_torch)

    # Вывод результатов теста
    print(f"Тест с параметрами:")
    print(f"  - in_channels: {in_channels}")
    print(f"  - out_channels: {out_channels}")
    print(f"  - kernel_size: {kernel_size}")
    print(f"  - stride: {stride}")
    print(f"  - padding: {padding}")
    print(f"  - dilation: {dilation}")
    print(f"  - groups: {groups}")
    print(f"  - bias: {bias}")
    print(f"  - padding_mode: {padding_mode}")
    print(f"\nРезультат транспонированной свертки совпадает: {are_results_equal}")
    print(f"\n{'-' * 75}\n")


In [71]:
def test_transpose_params():
    # параметры тестирования для первой группы
    run_transpose_test(in_channels=3, out_channels=2, kernel_size=3, stride=(2, 2), padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

    # параметры тестирования для второй группы
    run_transpose_test(in_channels=4, out_channels=2, kernel_size=(2, 3), stride=(2, 3), padding=(1, 1), dilation=1, groups=1, bias=True, padding_mode='replicate')


In [72]:
test_transpose_params()

Тест с параметрами:
  - in_channels: 3
  - out_channels: 2
  - kernel_size: 3
  - stride: (2, 2)
  - padding: 0
  - dilation: 1
  - groups: 1
  - bias: True
  - padding_mode: zeros

Результат транспонированной свертки совпадает: True

---------------------------------------------------------------------------

Тест с параметрами:
  - in_channels: 4
  - out_channels: 2
  - kernel_size: (2, 3)
  - stride: (2, 3)
  - padding: (1, 1)
  - dilation: 1
  - groups: 1
  - bias: True
  - padding_mode: replicate

Результат транспонированной свертки совпадает: False

---------------------------------------------------------------------------

